In [38]:
import requests
import pandas as pd
import json
import os

In [28]:
# Importing dataframe

dfs = pd.read_json('citybike_station_Como.json')[['station_name','latitude','longitude']]
dfs

,station_name,latitude,longitude
0,02 - Funicolare,45.817493,9.082644
1,03 - Municipio,45.809697,9.085078
2,04 - RFI San Giovanni,45.808697,9.073456
3,05 - Sant'Abbondio,45.802500,9.080200
4,06 - FN Borghi,45.804793,9.090773
5,07 - Vittoria,45.806006,9.085260
6,08 - Stadio,45.814664,9.073082
7,09 - Villa Olmo,45.818164,9.066306
8,10 - Santa Chiara,45.801285,9.086487
9,13 - Cacciatori delle Alpi,45.810234,9.077003


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [67]:
# Set the key

API_key = os.environ["FOURSQUARE_API_KEY"]

# Function to find points of interest in Foursquare of a specific category within a set distance from bike stations

def POI_fsq(station, latitudes, longitudes, radius, limit, category_ids):
    
    fields = 'fsq_id,name,popularity,price,rating'
    list_POI = []
    
    # Return station name and lists of points of interest
    
    for stat,lat,long in zip(station,latitudes,longitudes):
        url = f'https://api.foursquare.com/v3/places/search?ll={lat},{long}&radius={radius}&categories={category_ids}&limit={limit}&fields={fields}'
        headers = {"accept": "application/json",
                   "Authorization": API_key}
        response = requests.get(url, headers=headers).json()['results']
        list_POI.append([stat,response])
   
    # Convert to dataframe
    
    df = pd.DataFrame(list_POI,columns = ['station_name','results'])

    # create row for each match of bike station and points of interest
    
    df = df.explode('results').reset_index(drop=True) 
    df = pd.concat([df,pd.json_normalize(df['results'])],axis=1).drop('results',axis=1)
    return df


In [69]:
# Category - Dining and Drinking (sourse - https://location.foursquare.com/places/docs/categories)

df_dinning_drinks = POI_fsq(station = dfs['station_name'], latitudes = dfs['latitude'],
                            longitudes = dfs['longitude'], radius = 1000, limit = 50, 
                            category_ids = '13000')

df_dinning_drinks

,station_name,fsq_id,name,popularity,price,rating
0,02 - Funicolare,4d74eaae0a22236aa7dd3b02,Ristorante Pizzeria Funicolare,0.995358,2.0,8.5
1,02 - Funicolare,4bf982ce8d30d13a2d590218,La Vita è Bella Ristorante - Insalateria,0.990716,2.0,8.7
2,02 - Funicolare,4d2e0c96ad3a8cfab08890bd,Ristorante Il Carrettiere,0.957725,3.0,8.6
3,02 - Funicolare,55c6322e498e71e08d7b77b7,Gelateria I Tre Santi,0.983919,1.0,8.5
4,02 - Funicolare,4c0291adf423a593f8eece16,Ristorante Pizzeria Fontana D'Oro,0.993866,3.0,8.2
...,...,...,...,...,...,...
812,11 - Como Nord Lago,51d5b9b7498ea93b532c5396,Caffè Monti,0.995524,1.0,6.9
813,11 - Como Nord Lago,4d272805849f3704a8186641,Il Caffè dei Viaggiatori,0.985908,2.0,7.4
814,11 - Como Nord Lago,4c48a573417b20a1d4afe1a9,L'Antica Riva,0.970822,3.0,6.8
815,11 - Como Nord Lago,5729cfa1498e3209370103f2,Il Farcito,0.979775,3.0,7.3


In [40]:
# Category  - Landmarks and Outdoors

df_landmarks_outdoors = POI_fsq(station = dfs['station_name'], latitudes = dfs['latitude'],
                            longitudes = dfs['longitude'], radius = 1000, limit = 50, 
                            category_ids = '16000')

df_landmarks_outdoors

,station_name,fsq_id,name,popularity,rating
0,02 - Funicolare,4db6b6ec43a1369cb5eb542f,Piazza del Duomo,0.976956,8.7
1,02 - Funicolare,4be5e56b2457a593af2eac15,Piazza Cavour,0.995855,8.3
2,02 - Funicolare,55b63156498e872b9236ed16,Life Electric,0.990406,7.9
3,02 - Funicolare,4e0460d47d8b00eb78c10f0e,Monumento ai Caduti,0.991711,8.2
4,02 - Funicolare,4c4b16b75609c9b6321ea590,Piazza Alessandro Volta,0.997347,7.8
...,...,...,...,...,...
553,11 - Como Nord Lago,ff3f9187cb8e448cbf7c73b0,Evento storico,0.652174,NaN
554,11 - Como Nord Lago,b82bd02b4b09467c3cbb2ca6,Campisani Marcello,NaN,NaN
555,11 - Como Nord Lago,875d81a7bde04f8c8f50053b,Piazza Cavour SRL,NaN,NaN
556,11 - Como Nord Lago,a054d7ff120541a8ac9d8b62,Novocomum,NaN,NaN


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Get the top 10 restaurants according to their rating

In [54]:
# Set the key

API_key_yelp = os.environ["YELP_FUSION_API_KEY"]

# Function to find points of interest in Yelp of a specific category within a set distance from bike stations

def POI_yelp(station,latitudes,longitudes,radius,limit,categories):
    
    list_POI = []
    
    # Return station name and lists of points of interest
    
    for stat,lat,long in zip(station,latitudes,longitudes):
        url = f'https://api.yelp.com/v3/businesses/search?latitude={lat}&longitude={long}&radius={radius}&limit={limit}&categories={categories}'
        headers = {"accept": "application/json","Authorization": 'bearer ' + API_key_yelp}
        response = requests.get(url, headers=headers).json()['businesses']
        list_POI.append([stat,response])
    
    # Convert to dataframe
    
    df = pd.DataFrame(list_POI,columns = ['station_name','businesses'])

    # create row for each match of bike station and points of interest 
    
    df = df.explode('businesses').reset_index(drop=True) 
    df = pd.concat([df,pd.json_normalize(df['businesses'])],axis=1).drop('businesses',axis=1)
    return df


In [55]:
# Category  - Restaurants and Bars (sourse - https://docs.developer.yelp.com/docs/resources-categories)

df_restaurants_bars = POI_yelp(station = dfs['station_name'], latitudes = dfs['latitude'],
                              longitudes = dfs['longitude'], radius = 1000,
                              limit = 50, categories = 'restaurants,bars')

df_restaurants_bars

,station_name,id,alias,name,image_url,is_closed,url,review_count,categories,rating,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,02 - Funicolare,SkFge_8f5l0Xb3tYsbU09Q,borgo-santagostino-como,Borgo Sant'Agostino,https://s3-media3.fl.yelpcdn.com/bphoto/dqDmb8...,False,https://www.yelp.com/biz/borgo-santagostino-co...,10,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,...,45.817452,9.083010,Piazza Alcide de Gasperi 8,,,Como,22100,IT,CO,"[Piazza Alcide de Gasperi 8, 22100 Como, Italy]"
1,02 - Funicolare,-u6IjWcIiMzrophJ-_hbqQ,fontana-d-oro-como,Fontana D'oro,https://s3-media3.fl.yelpcdn.com/bphoto/buD-Ol...,False,https://www.yelp.com/biz/fontana-d-oro-como?ad...,15,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,...,45.816701,9.082827,Via Dionigi da Parravicino 3,None,None,Como,22100,IT,LC,"[Via Dionigi da Parravicino 3, 22100 Como, Italy]"
2,02 - Funicolare,uCu0x_5ZoECf7mGbyowflQ,ox-pub-and-grill-como,Ox Pub And Grill,https://s3-media3.fl.yelpcdn.com/bphoto/E5z6e2...,False,https://www.yelp.com/biz/ox-pub-and-grill-como...,5,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",4.5,...,45.817537,9.082812,Piazza Alcide de Gasperi 6,,None,Como,22100,IT,CO,"[Piazza Alcide de Gasperi 6, 22100 Como, Italy]"
3,02 - Funicolare,YezGpl7mHwl4P5qLg_PYZw,la-vita-è-bella-como,La Vita è Bella,https://s3-media4.fl.yelpcdn.com/bphoto/kLhXRT...,False,https://www.yelp.com/biz/la-vita-%C3%A8-bella-...,30,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",4.5,...,45.815866,9.083368,Piazza Domenico Croggi 4,,None,Como,22100,IT,CO,"[Piazza Domenico Croggi 4, 22100 Como, Italy]"
4,02 - Funicolare,tz_DFTykt3Yoz00g8RvqLg,piazza-roma-como,Piazza Roma,https://s3-media3.fl.yelpcdn.com/bphoto/GEDcoE...,False,https://www.yelp.com/biz/piazza-roma-como?adju...,22,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",5.0,...,45.813025,9.083409,Piazza Roma 43,,None,Como,22100,IT,CO,"[Piazza Roma 43, 22100 Como, Italy]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,11 - Como Nord Lago,lQtTVNYNfWdCs0NvfbkafQ,rino-como,Rino,https://s3-media4.fl.yelpcdn.com/bphoto/RLJdxa...,False,https://www.yelp.com/biz/rino-como?adjust_crea...,7,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,...,45.811439,9.080880,Via Vitani 3,,,Como,22100,IT,CO,"[Via Vitani 3, 22100 Como, Italy]"
701,11 - Como Nord Lago,ZFJbfNQmkuZhd0eeqhbkxg,cornobobo-como,Cornobobo,https://s3-media1.fl.yelpcdn.com/bphoto/RgbYtm...,False,https://www.yelp.com/biz/cornobobo-como?adjust...,1,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",5.0,...,45.814903,9.084760,Piazza Giovanni Amendola 1,,None,Como,22100,IT,CO,"[Piazza Giovanni Amendola 1, 22100 Como, Italy]"
702,11 - Como Nord Lago,xoYXciu0Gj0wC4kd0HCQ8A,la-cucina-di-stella-como,La Cucina di Stella,https://s3-media4.fl.yelpcdn.com/bphoto/i-dVys...,False,https://www.yelp.com/biz/la-cucina-di-stella-c...,11,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,...,45.809313,9.082647,Via Indipendenza 56,,,Como,22100,IT,CO,"[Via Indipendenza 56, 22100 Como, Italy]"
703,11 - Como Nord Lago,GCqoJrAw_OMyHdLfnP9dMw,tigella-bella-como,Tigella Bella,https://s3-media2.fl.yelpcdn.com/bphoto/GJyAHJ...,False,https://www.yelp.com/biz/tigella-bella-como?ad...,5,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",5.0,...,45.808530,9.083040,Via Natta 35,,None,Como,22100,IT,CO,"[Via Natta 35, 22100 Como, Italy]"


In [56]:
# Category  - Landmarks & Historical Buildings, Arts & Entertainment, and Active life

df_landmarks_arts_active = POI_yelp(station = dfs['station_name'], latitudes = dfs['latitude'],
                              longitudes = dfs['longitude'], radius = 1000,
                              limit = 50, categories = 'landmarks,arts,active')

df_landmarks_arts_active

,station_name,id,alias,name,image_url,is_closed,url,review_count,categories,rating,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,02 - Funicolare,G0fPt9S422e1tIpzCz_KVQ,vintage-como,Vintage,https://s3-media4.fl.yelpcdn.com/bphoto/Kdb2-e...,False,https://www.yelp.com/biz/vintage-como?adjust_c...,8.0,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,...,45.811196,9.080020,Via Olginati 14,,,Como,22100,IT,CO,"[Via Olginati 14, 22100 Como, Italy]"
1,02 - Funicolare,Et_x6wo5FXP6HOr_7yE7Eg,stunning-bike-co-tours-bike-rental-e-shipments...,Stunning Bike Co-Tours | Bike Rental & Shipmen...,https://s3-media3.fl.yelpcdn.com/bphoto/yc4-Ts...,False,https://www.yelp.com/biz/stunning-bike-co-tour...,1.0,"[{'alias': 'bikerentals', 'title': 'Bike Renta...",5.0,...,45.816720,9.083570,Via Coloniola 37,None,,Como,22100,IT,CO,"[Via Coloniola 37, 22100 Como, Italy]"
2,02 - Funicolare,vNV-oaBbrtWie6rIVTyNiQ,il-sorso-como,Il Sorso,https://s3-media2.fl.yelpcdn.com/bphoto/LsoLmd...,False,https://www.yelp.com/biz/il-sorso-como?adjust_...,5.0,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.0,...,45.811867,9.079779,Piazza Alessandro Volta 8,,,Como,22100,IT,CO,"[Piazza Alessandro Volta 8, 22100 Como, Italy]"
3,02 - Funicolare,_vrKq93CwV6VKF2GYBECXw,aero-club-como,Aero Club,https://s3-media4.fl.yelpcdn.com/bphoto/yUS_us...,False,https://www.yelp.com/biz/aero-club-como?adjust...,2.0,"[{'alias': 'active', 'title': 'Active Life'}]",5.0,...,45.813732,9.070795,Viale Masia 44,,,Como,22100,IT,CO,"[Viale Masia 44, 22100 Como, Italy]"
4,02 - Funicolare,-UIAO_kw7FEWNYftCNVOSg,comer-dom-como,Comer Dom,https://s3-media2.fl.yelpcdn.com/bphoto/e4YbH5...,False,https://www.yelp.com/biz/comer-dom-como?adjust...,5.0,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",4.5,...,45.811611,9.083060,Piazza del Duomo,,,Como,22100,IT,CO,"[Piazza del Duomo, 22100 Como, Italy]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,11 - Como Nord Lago,9pPgkn-LO1VzxABREyF8_Q,palazzo-cernezzi-como,Palazzo Cernezzi,https://s3-media1.fl.yelpcdn.com/bphoto/g6x1ev...,False,https://www.yelp.com/biz/palazzo-cernezzi-como...,1.0,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",5.0,...,45.809930,9.084789,Via Vittorio Emanuele II 97,,,Como,22100,IT,CO,"[Via Vittorio Emanuele II 97, 22100 Como, Italy]"
117,11 - Como Nord Lago,ka960ldqOUnClSCfHDW1CA,museo-a-volta-como,Museo A Volta,https://s3-media1.fl.yelpcdn.com/bphoto/HlhFt0...,False,https://www.yelp.com/biz/museo-a-volta-como?ad...,1.0,"[{'alias': 'museums', 'title': 'Museums'}]",4.0,...,45.814587,9.075372,Lungolago Mafalda di Savoia,,,Como,22100,IT,CO,"[Lungolago Mafalda di Savoia, 22100 Como, Italy]"
118,11 - Como Nord Lago,gosHgKFlbvSo9WE8ZtNeWg,giardini-tempio-voltiano-como,Giardini Tempio Voltiano,https://s3-media1.fl.yelpcdn.com/bphoto/GXRNy_...,False,https://www.yelp.com/biz/giardini-tempio-volti...,1.0,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,...,45.814651,9.074615,Viale Puecher 4,,,Como,22100,IT,CO,"[Viale Puecher 4, 22100 Como, Italy]"
119,11 - Como Nord Lago,IbYnKX2X1p5w-O3YXEPzRA,san-fedele-basilica-como,San Fedele Basilica,https://s3-media4.fl.yelpcdn.com/bphoto/7pSzGK...,False,https://www.yelp.com/biz/san-fedele-basilica-c...,1.0,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",2.0,...,45.809524,9.084191,Via Vittorio Emanuele II 94,,,Como,22100,IT,CO,"[Via Vittorio Emanuele II 94, 22100 Como, Italy]"


In [63]:
# Removing unnecessary columns 

df_restaurants_bars = df_restaurants_bars[['station_name', 'id', 'name',
       'review_count', 'rating', 'price']]

df_restaurants_bars

,station_name,id,name,review_count,rating,price
0,02 - Funicolare,SkFge_8f5l0Xb3tYsbU09Q,Borgo Sant'Agostino,10,4.5,€€
1,02 - Funicolare,-u6IjWcIiMzrophJ-_hbqQ,Fontana D'oro,15,4.5,€€
2,02 - Funicolare,uCu0x_5ZoECf7mGbyowflQ,Ox Pub And Grill,5,4.5,NaN
3,02 - Funicolare,YezGpl7mHwl4P5qLg_PYZw,La Vita è Bella,30,4.5,€€
4,02 - Funicolare,tz_DFTykt3Yoz00g8RvqLg,Piazza Roma,22,5.0,NaN
...,...,...,...,...,...,...
700,11 - Como Nord Lago,lQtTVNYNfWdCs0NvfbkafQ,Rino,7,4.5,€€€
701,11 - Como Nord Lago,ZFJbfNQmkuZhd0eeqhbkxg,Cornobobo,1,5.0,NaN
702,11 - Como Nord Lago,xoYXciu0Gj0wC4kd0HCQ8A,La Cucina di Stella,11,4.5,€€€
703,11 - Como Nord Lago,GCqoJrAw_OMyHdLfnP9dMw,Tigella Bella,5,5.0,NaN


In [65]:
# Removing unnecessary columns

df_landmarks_arts_active = df_landmarks_arts_active[['station_name', 'id', 'name',
       'review_count', 'rating']]

df_landmarks_arts_active

,station_name,id,name,review_count,rating
0,02 - Funicolare,G0fPt9S422e1tIpzCz_KVQ,Vintage,8.0,4.5
1,02 - Funicolare,Et_x6wo5FXP6HOr_7yE7Eg,Stunning Bike Co-Tours | Bike Rental & Shipmen...,1.0,5.0
2,02 - Funicolare,vNV-oaBbrtWie6rIVTyNiQ,Il Sorso,5.0,4.0
3,02 - Funicolare,_vrKq93CwV6VKF2GYBECXw,Aero Club,2.0,5.0
4,02 - Funicolare,-UIAO_kw7FEWNYftCNVOSg,Comer Dom,5.0,4.5
...,...,...,...,...,...
116,11 - Como Nord Lago,9pPgkn-LO1VzxABREyF8_Q,Palazzo Cernezzi,1.0,5.0
117,11 - Como Nord Lago,ka960ldqOUnClSCfHDW1CA,Museo A Volta,1.0,4.0
118,11 - Como Nord Lago,gosHgKFlbvSo9WE8ZtNeWg,Giardini Tempio Voltiano,1.0,5.0
119,11 - Como Nord Lago,IbYnKX2X1p5w-O3YXEPzRA,San Fedele Basilica,1.0,2.0


In [82]:
# Saving dataframe to csv file

df_restaurants_bars.to_csv(path_or_buf='yelp_df_restaurants_bars.csv',index=False)

In [75]:
df_landmarks_arts_active.to_csv(path_or_buf='yelp_df_landmarks_arts_active.csv',index=False)

In [76]:
df_dinning_drinks.to_csv(path_or_buf='fsq_df_dinning_drinks.csv',index=False)

In [77]:
df_landmarks_outdoors.to_csv(path_or_buf='fsq_df_landmarks_outdoors.csv',index=False)

# Comparing Results

In [57]:
# Calculating the percentage calculation of not-null values in columns

print(df_dinning_drinks.count()/df_dinning_drinks.shape[0])
print()
print(str(df_dinning_drinks.shape[0]) + ' rows')
print('Mean not null percentage: '+ str((df_dinning_drinks.count()/df_dinning_drinks.shape[0]).mean()))

station_name    1.000000
fsq_id          1.000000
name            1.000000
popularity      0.908201
price           0.785802
rating          0.725826
dtype: float64

817 rows
Mean not null percentage: 0.9033047735618115


In [58]:
print(df_landmarks_outdoors.count()/df_landmarks_outdoors.shape[0])
print()
print(str(df_landmarks_outdoors.shape[0]) + ' rows')
print('Mean not null percentage: '+ str((df_landmarks_outdoors.count()/df_landmarks_outdoors.shape[0]).mean()))

station_name    1.000000
fsq_id          1.000000
name            1.000000
popularity      0.654122
rating          0.152330
dtype: float64

558 rows
Mean not null percentage: 0.7612903225806452


In [70]:
print(df_restaurants_bars.count()/df_restaurants_bars.shape[0])
print()
print(str(df_restaurants_bars.shape[0]) + ' rows')
print('Mean not null percentage: '+ str((df_restaurants_bars.count()/df_restaurants_bars.shape[0]).mean()))

station_name    1.000000
id              1.000000
name            1.000000
review_count    1.000000
rating          1.000000
price           0.582979
dtype: float64

705 rows
Mean not null percentage: 0.9304964539007092


In [71]:
print(df_landmarks_arts_active.count()/df_landmarks_arts_active.shape[0])
print()
print(str(df_landmarks_arts_active.shape[0]) + ' rows')
print('Mean not null percentage: '+ str((df_landmarks_arts_active.count()/df_landmarks_arts_active.shape[0]).mean()))

station_name    1.000000
id              0.991736
name            0.991736
review_count    0.991736
rating          0.991736
dtype: float64

121 rows
Mean not null percentage: 0.9933884297520661


Which API provided you with more complete data? Provide an explanation. 

Based on the analysis, it appears that Foursquare has an advantage in terms of data coverage and information about the popularity of places for both restaurants and bars as well as points of interest (landmarks, historical buildings, arts, entertainment, and active life). However, Yelp is superior when it comes to high-quality rating information for restaurants and bars, while Foursquare may have limitations in this area.

If we are primarily interested in completeness and data coverage, especially for points of interest, Foursquare may be the better choice. However, if we prioritize having accurate and detailed rating information for restaurants and bars and the price level is not particularly important, Yelp might be the preferred API.

In our specific case, the city Como is a popular tourist destination, and it is interesting for its sights. In this context, received data from Foursquare seems like the more suitable choice to explore.

Get the top 10 restaurants according to their rating

In [80]:
# Top 10 restaurants(bars) according to rating in Foursquare

df_dinning_drinks.sort_values('rating',ascending=False).drop_duplicates('fsq_id').head(10)

,station_name,fsq_id,name,popularity,price,rating
773,11 - Como Nord Lago,4c2d14e2260bc928c16519d3,Gelateria Lariana,0.999005,1.0,9.2
153,05 - Sant'Abbondio,4bdc961dc79cc928cd3d87e9,Fresco Cocktail Shop,0.969828,2.0,9.1
260,07 - Vittoria,4e17eeed183880768f3c1cda,Pasticceria Vittoria,0.984416,2.0,9.0
300,08 - Stadio,4d3b75de49cb236a8383c447,Il Birrivico,0.990882,2.0,8.9
719,17 - Cernobbio,4c4f35718b5520a12a511f04,Trattoria Del Glicine,0.900875,NaN,8.8
795,11 - Como Nord Lago,500be89fe4b0e57c7659635a,Trattoria dei Bracconieri,0.969622,2.0,8.8
513,14 - Pirellino,4bf982ce8d30d13a2d590218,La Vita è Bella Ristorante - Insalateria,0.990716,2.0,8.7
512,14 - Pirellino,4f06c2b2e4b0aa2dc48d66ad,Dolciamo Pasticceria,0.968667,1.0,8.7
208,06 - FN Borghi,4fd392ede4b0f961a7c25a32,Ristorante La Colombetta,0.956399,3.0,8.7
629,01 - Cavour,5159a690e4b0ac344fc53cc5,I Tigli in Theoria,0.943302,3.0,8.7


In [79]:
# Top 10 restaurants(bars) according to rating in Yelp

df_restaurants_bars.sort_values(['rating','review_count'],ascending=False).drop_duplicates('id').head(10)

,station_name,id,name,review_count,rating,price
4,02 - Funicolare,tz_DFTykt3Yoz00g8RvqLg,Piazza Roma,22,5.0,NaN
30,02 - Funicolare,7FIkZMVmUaNOhKS_I70dxw,Gelateria Lariana,16,5.0,€
17,02 - Funicolare,bJjLbHnOKmWsSU5HC4McUQ,Fresco Cocktail Shop,9,5.0,€€
34,02 - Funicolare,GCqoJrAw_OMyHdLfnP9dMw,Tigella Bella,5,5.0,NaN
627,12 - Tavernola,JdcxahxkCzO1Fe3IOEKPxg,Ristorante Materia,5,5.0,NaN
5,02 - Funicolare,nuncT6hfZtInOz9oRzYU6g,Yoko Sushi,4,5.0,NaN
32,02 - Funicolare,83UsMRHeKo0cGtvkD5uYIg,La Cucina di Elsa,4,5.0,NaN
48,02 - Funicolare,nr4KCvtmxTBcyKMKpXiihQ,Natta Café,4,5.0,NaN
27,02 - Funicolare,tl-aLY5CFYu4B7gW_y_TYg,Gelateria AL Molo,3,5.0,NaN
76,03 - Municipio,12G3BfEnwUkpbx20FCi5oA,Karma Como,3,5.0,NaN
